In [138]:
import numpy as np
import six
import sys
import os
import traceback
import re
import pickle
from copy import deepcopy

from chainer import cuda
from context2vec.common.context_models import Toks
from context2vec.common.model_reader import ModelReader
import sklearn
import pandas as pd
import logging
from scipy.stats import spearmanr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
import math
import collections
import argparse


[nltk_data] Downloading package stopwords to /home/ql261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [105]:
# from sklearn.preprocessing import normalize
# a=np.array([1.23,2.232,3.2])
# print(a/np.sqrt((a*a).sum()))
# print (normalize([a]))

[0.30067397 0.54561326 0.78224123]
[[0.30067397 0.54561326 0.78224123]]


In [41]:
def produce_top_n_simwords(w_filter,context_embed,n_result,index2word,debug=False):
        #assume that w_filter is already normalized
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
        similarity_scores=[]
        print('producing top {0} simwords'.format(n_result))
        similarity = (w_filter.dot(context_embed)+1.0)/2
        top_words_i=[]
        top_words=[]
        count = 0
        for i in (-similarity).argsort():
                    if xp.isnan(similarity[i]):
                        continue
                    if debug==True:
                        print('{0}: {1}'.format(str(index2word[int(i)]), str(similarity[int(i)])))
                    count += 1
                    top_words_i.append(int(i))
                    top_words.append(index2word[int(i)])
                    similarity_scores.append(float(similarity[int(i)]))
                    if count == n_result:
                        break

        top_vec=w_filter[top_words_i,:]
        return top_vec,xp.array(similarity_scores),top_words
    
def top_mutual_sim(top_vec,similarity_scores):

    #normalize the top_vec
    s = xp.sqrt((top_vec * top_vec).sum(1))
    s[s==0.] = 1.
    top_vec /= s.reshape((s.shape[0], 1))
    
    # substitutes' similarity to sentence (similarity_scores) as weight matrix to mutual similarity
    max_score=similarity_scores[0]
    similarity_scores=np.array(similarity_scores)
    sim_weights=(similarity_scores+similarity_scores.reshape(len(similarity_scores),1))/2.0
    #weighted by the maximum score in the substitutes (highre max score means the context is more certain about the substitutes)
    sim_weights=(sim_weights/float(sum(sum(sim_weights))))*max_score
    # dot product weighted by substitute probability (sim_weights)
    inf_score=sum(sum(top_vec.dot(top_vec.T)*sim_weights))
    return inf_score

def top_cluster_density(top_vec,similarity_scores):
    #normalize the top_vec
    s = xp.sqrt((top_vec * top_vec).sum(1))
    s[s==0.] = 1.
    top_vec = top_vec/ s.reshape((s.shape[0], 1))
    
    #perform the centroid
    max_score=similarity_scores[0]
    similarity_scores=np.array(similarity_scores).reshape(len(similarity_scores),1)/sum(similarity_scores)
    centroid_vector=sum(top_vec*similarity_scores)
    # average of cosine distance to the centroid,weighted by max scores
    inf_score=sum(top_vec.dot(centroid_vector))/len(top_vec)*max_score
    return inf_score

In [133]:
def load_w2salience(w2salience_f,weight_type):
    w2salience={}
    with open(w2salience_f) as f:
        for line in f:
            if line.strip()=='':
                continue
            w,w_count,s_count=line.strip().split('\t')
            if weight_type==INVERSE_W_FREQ:
                w2salience[w]=1/float(w_count)
            elif weight_type==INVERSE_S_FREQ:
                w2salience[w]=math.log(1+84755431/float(s_count))
    return w2salience

def skipgram_context(model,words,pos,weight=None,w2entropy=None):
    context_wvs=[]
    weights=[]
    for i,word in enumerate(words):
        if i != pos: #surroudn context words
            try:
                if weight ==LDA:
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(1/(w2entropy[word]+1.0))
                        context_wvs.append(model[word])
                elif weight in [INVERSE_W_FREQ,INVERSE_S_FREQ]:
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(w2entropy[word])
                        context_wvs.append(model[word])
                else:
                    #equal weights per word
                    context_wvs.append(model[word])
                    weights.append(1.0)
            except KeyError as e:
                print ('==warning==: key error in context {0}'.format(e))
    print ('per word weights',weights)
    context_embed=sum(np.array(context_wvs)*np.array(weights).reshape(len(weights),1))#/sum(weights)
    return sum(weights),context_embed #  will be normalized later

def lg_model_out_w2v(top_words,w_target,word2index_target):
        # lg model substitutes in skipgram embedding
        top_vec=[]
        index_list=[]
        for i,word in enumerate(top_words):
            try :
                top_vec.append(w_target[word2index_target[word]])
                index_list.append(i)
            except KeyError as e:
                print (e)
        return xp.stack(top_vec),index_list
    
def context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    #produce context representation and infromative score for each context
    test_s=test_s.replace(test_w, ' '+test_w+' ')
    print(test_s)
    words=test_s.split()
    pos=words.index(test_w)
    
    score=1.0 #default score
    
    # Decide on the model
    if model_type=='context2vec':
        context_embed= model.context2vec(words, pos)
        
    elif model_type=='skipgram':
        score,context_embed=skipgram_context(model,words,pos,weight,w2entropy)
        context_embed_out=xp.array(context_embed)
        
    elif model_type=='context2vec-skipgram':
        # context2vec substitutes in skipgram space
        context_embed= model.context2vec(words, pos)
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        top_vec,index_list=lg_model_out_w2v(top_words,w_target,word2index_target) 
        sim_scores=sim_scores[index_list] #weighted by substitute probability
        print (type(top_vec),type(sim_scores))
        if weight==SUBSTITUTE_PROB:
            context_embed_out=xp.array(sum(top_vec*sim_scores.reshape(len(sim_scores),1)))
        else:
            context_embed_out=xp.array(sum(top_vec*((sim_scores/sum(sim_scores)).reshape(len(sim_scores),1))))

    else:
        print ('model type {0} not recognized'.format(model_type))
        sys.exit(1)
        
    print('context_embed original', context_embed[:10])
    print ('context_embed_out',context_embed_out[:10])
    print ('3',type(context_embed_out))

    #decide on weight per sentence
    print ('weight mode',weight)
    if weight==TOP_MUTUAL_SIM:
#         if word2index_target==None: #not context2vec-skipgram
#             context2vec word embedding space neighbours
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        #skipgram word embedding space neighbours when context2vec-skipgram
        score=top_mutual_sim(top_vec,sim_scores)
    elif weight==TOP_CLUSTER_DENSITY:
#         if word2index_target==None: #not context2vec-skipgram
#             context2vec word embedding space neighbours
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        score=top_cluster_density(top_vec,sim_scores)
    elif weight==SUBSTITUTE_PROB:
        score=sum(sim_scores)
        print ('substitute prob score',score)
    elif weight=='learned':
        print ('learned not implemented')
    elif weight=='gaussian':
        print ('gaussian not implemented')
    elif weight ==False or weight in [LDA,INVERSE_S_FREQ,INVERSE_W_FREQ]:
        score=score
    else:
        print ('weight mode {0} not recognized'.format(weight))

    return score,context_embed_out

def additive_model(test_ss,test_w, model_type,model,n_result,w_filter,index2word,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None,f_w=None):
    #produce context representation across contexts using weighted average
    context_out=[]
    context_weights=[]
    for test_s in test_ss:
        test_s=test_s.strip()
        #produce context representation with scores
        score,context_embed=context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
        context_out.append(context_embed)
        print ('context_embedtype',type(context_embed))
        context_weights.append(score)
    
    
    print ('context_weights',context_weights)
    #sum representation across contexts
    context_out=xp.stack(context_out)
    
    
    if model_type=='skipgram' or weight==SUBSTITUTE_PROB:
        # context representation by weighted sum of all context words in all contexts
        context_avg=sum(context_out)/sum(context_weights)
    else:
        norm_weights=xp.array(context_weights).reshape(len(context_weights),1)/float(sum(context_weights))
        if f_w!=None:
            f_w.write(','.join([str(i[0]) for i in norm_weights])+'\n')
        print ('normalized weight: \n  {0}'.format(norm_weights))
        # context represenatation by weighted sum of contexts
        context_avg=sum(norm_weights*context_out)
    
    
    # check new embedding neighbours

    print('producing top {0} words for new embedding'.format(n_result))
    if index2word_target==None:
        top_vec,scores,top_words=produce_top_n_simwords(w_filter,context_avg,n_result,index2word,debug=True)
    else:
        #print the target space neighbours for context2vec-skipgram
        top_vec,scores,top_words=produce_top_n_simwords(w_target,context_avg,n_result,index2word_target,debug=True)
    
    return context_avg




In [134]:
def filter_w(w,word2index,index2word):
    #filter out words with no letters in, and stopwords
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    index2word_filter={}
    word2index_filter={}
    index_filter2index=[]
    counter=0
    for word in word2index:
            if word not in stopw:
                    index_filter2index.append(word2index[word])
                    word2index_filter[word]=counter
                    index2word_filter[counter]=word
                    counter+=1
    w_filter= w[index_filter2index,:]
    return w_filter,word2index_filter,index2word_filter

def rm_stopw_context(model):
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    
    model2={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
    return model2




In [135]:
def preprocess_nonce(sent):
    
    sents_out=[]
    
    sent=sent.lower()
    results=re.finditer('___ ',sent)
    matches=[m for m in results]
    for i in range(len(matches)):
        sent_masked=sent
        matches_mask=[(m2.start(0),m2.end(0)) for i2,m2 in enumerate(matches) if i2!=i]
        matches_mask=sorted(matches_mask, key=lambda x:x[0],reverse=True)
        for m in matches_mask:
            sent_masked=sent_masked[:m[0]]+sent_masked[m[1]:]
        sents_out.append(sent_masked+' .')
    return sents_out

def eval_nonce(nonce_data_f,context_model,model_w2v,model_type,n_result,w,index2word,word2index,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
        ranks = []
        mrr = 0.0
        data=pd.read_csv(os.path.join(nonce_data_f),delimiter='\t',header=None,comment='#')
        c = 0
        for index, row in data.iterrows():
            if index>100 and index%100==0:
                print (index)
            sents=preprocess_nonce(row[1])
            nonce=row[0]
            if nonce not in model_w2v:
                print ('{0} not known'.format(nonce))
                continue
            #compute context representation
            if model_type=='context2vec-skipgram?skipgram':
                    #context2vevc                
                    context_avg_1=additive_model(sents,'___', model_type.split('?')[0],context_model[0],n_result,w[0],index2word[0],weight[0],w2entropy[0],w_target[0],word2index_target[0],index2word_target[0])
                    print ('context2vec avg embed',context_avg_1[:10])
                    context_avg_2=additive_model(sents,'___', model_type.split('?')[1],context_model[1],n_result,w[1],index2word[1],weight[1],w2entropy[1],w_target[1],word2index_target[1],index2word_target[1])
                    print ('skipgram avg embed', context_avg_2[:10])
                    context_avg=(context_avg_1+context_avg_2)/2
                    print ('context2vec avg out', context_avg[:10])
                    #compute probe embeddings in skipgram space
                    w_out=w[1]
                    w_target_out=w_target[1]
                    word2index_out=word2index[1]
                    word2index_target_out=word2index_target[1]
                    
            else:
                    
                    context_avg=additive_model(sents,'___', model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
                    print ('context avg out', context_avg[:10])
                    w_out=w
                    w_target_out=w_target
                    word2index_out=word2index
                    word2index_target_out=word2index_target
            
            if xp==cuda.cupy:
                context_avg=xp.asnumpy(context_avg)
            print ('vector norm: {0}'.format(np.linalg.norm(context_avg)))
            
            
            # MRR Rank calculation
            nns=model_w2v.similar_by_vector(context_avg,topn=len(model_w2v.wv.vocab))

            rr = 0
            n = 1
            for nn in nns:
                word = nn[0]
                if word == nonce:
                    print (word)
                    rr = n
                    ranks.append(rr)
                else:
                  n+=1

            if rr != 0:
                mrr+=float(1)/float(rr)	
            print rr,mrr
            c+=1
        print ("Final MRR: ",mrr,c,float(mrr)/float(c))

        print ('mediam : {0}'.format(np.median(ranks)))
        return ranks
            


def eval_chimera(chimeras_data_f,context_model,model_type,n_result,w,index2word,word2index,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    chimeras_data_dir='/'.join(chimeras_data_f.split('/')[:-1])
    num_sent=chimeras_data_f.split('/')[-1].split('.')[1][1]
    print (chimeras_data_dir)
    print (num_sent)
    with open(chimeras_data_dir+'/weights_{0}_{1}_{2}'.format(num_sent,model_type,str(weight)),'w') as f_w:
        spearmans=[]
        data=pd.read_csv(os.path.join(chimeras_data_f),delimiter='\t',header=None)

        for index, row in data.iterrows():
            if index>100 and index%100==0:
                print (index)
            golds=[]
            model_predict=[]
            probes=[]
            #compute context representation
            sents=row[1].lower().split('@@')
            if model_type=='context2vec-skipgram?skipgram':
                    #context2vevc
                    context_avg_1=additive_model(sents,'___', model_type.split('?')[0],context_model[0],n_result,w[0],index2word[0],weight[0],w2entropy[0],w_target[0],word2index_target[0],index2word_target[0],f_w)
                    print ('context2vec avg embed',context_avg_1[:10])
                    context_avg_2=additive_model(sents,'___', model_type.split('?')[1],context_model[1],n_result,w[1],index2word[1],weight[1],w2entropy[1],w_target[1],word2index_target[1],index2word_target[1],f_w)
                    print ('skipgram avg embed', context_avg_2[:10])
                    context_avg=(context_avg_1+context_avg_2)/2
                    print ('context2vec avg out', context_avg[:10])
                    #compute probe embeddings in skipgram space
                    w_out=w[1]
                    w_target_out=w_target[1]
                    word2index_out=word2index[1]
                    word2index_target_out=word2index_target[1]
                    
            else:

                    context_avg=additive_model(sents,'___', model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target,f_w)
                    print ('context avg out', context_avg[:10])
                    w_out=w
                    w_target_out=w_target
                    word2index_out=word2index
                    word2index_target_out=word2index_target
            
            context_avg = context_avg / xp.sqrt((context_avg * context_avg).sum())

           
            
            #cosine similarity with probe embedding
            for gold,probe in zip(row[3].split(','),row[2].split(',')):
                try:
                    if word2index_target_out==None:
                        probe_w_vec=w_out[word2index_out[probe]]
                    else:
                        probe_w_vec=w_target_out[word2index_target_out[probe]]
                    probe_w_vec=probe_w_vec/xp.sqrt((probe_w_vec*probe_w_vec).sum())
                    cos=float(probe_w_vec.dot(context_avg))
                    if np.isnan(cos):
                        continue
                    else:
                        model_predict.append(cos)
                        golds.append(gold)
                        probes.append(probe)
                except KeyError as e:
                    print ("====warning key error for probe=====: {0}".format(e))
            print ('probes',probes)
            print ('gold',golds)
            print ('model_predict',model_predict)
            sp=spearmanr(golds,model_predict)[0]
            print ('spearman correlation is {0}'.format(sp))
            if not math.isnan(sp):
                spearmans.append(sp)
        print ("AVERAGE RHO:",float(sum(spearmans))/float(len(spearmans)))

In [139]:
TOP_MUTUAL_SIM='top_mutual_sim'
TOP_CLUSTER_DENSITY='top_cluster_density'
LDA='lda'
INVERSE_S_FREQ='inverse_s_freq'
INVERSE_W_FREQ='inverse_w_q'
SUBSTITUTE_PROB='substitute_prob'
WEIGHT_DICT={0:False,1:TOP_MUTUAL_SIM,2:LDA,3:INVERSE_S_FREQ,4:INVERSE_W_FREQ,5:TOP_CLUSTER_DENSITY, 6:SUBSTITUTE_PROB}


if __name__=="__main__":
    
    #params read in
    if sys.argv[0]=='/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py':
        
        data='./eval_data/data-chimeras/dataset.l2.fixed.test.txt.punct'
#         data='./eval_data/data-nonces/n2v.definitional.dataset.train.txt'
        weight=WEIGHT_DICT[0]
        gpu=1
#         ##context2vec
##         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params'
#         model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14'
        
        model_type='context2vec-skipgram'

####skipgram
        if model_type=='skipgram':
            model_param_file='../models/wiki_all.model/wiki_all.sent.split.model'
            model_type='skipgram'
    #         weight='inverse_w_freq'
    #         w2salience_f='../corpora/corpora/wiki.all.utf8.sent.split.tokenized.vocab'
    #         w2salience_f='../models/lda/w2entropy'
            n_result=20
            w2salience_f=None
        elif model_type=='context2vec-skipgram':
            model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14?../models/wiki_all.model/wiki_all.sent.split.model'
    #         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params?../models/wiki_all.model/wiki_all.sent.split.model'
            n_result=20
            w2salience_f=None
        elif model_type=='skipgram?context2vec-skipgram':
        #####skipgram?context2vec-skipgram
            model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14?../models/wiki_all.model/wiki_all.sent.split.model'
    #         weight='inverse_w_freq'
    #         w2salience_f='../corpora/corpora/wiki.all.utf8.sent.split.tokenized.vocab'
    #         w2salience_f='../models/lda/w2entropy'
            n_result=20
            w2salience_f=None
    
    else:
        

        parser = argparse.ArgumentParser(description='Process some integers.')
        parser.add_argument('--f',  type=str,
                            help='model_param_file',dest='model_param_file')
        parser.add_argument('--m', dest='model_type', type=str,
                            help='<model_type: context2vec; context2vec-skipgram (context2vec substitutes in skipgram space); context2vec-skipgram?skipgram (context2vec substitutes in skipgram space plus skipgram context words)>')
        parser.add_argument('--w', dest='weight', type=str, help='<weight:{0}>'.format (sys.argv[0],WEIGHT_DICT.items()))       
        parser.add_argument('--d', dest='data', type=str, help='data file')
        parser.add_argument('--g', dest='gpu',type=int, default=-1,help='gpu, default is -1')
        parser.add_argument('--ws', dest='w2salience_f',type=str, default=None,help='word2salience file, optional')

        args = parser.parse_args()
        
#         if len(sys.argv) < 6:
#             print >> sys.stderr, "Usage: {0} <model_param_file> <model_type: context2vec; context2vec-skipgram (context2vec substitutes in skipgram space); context2vec-skipgram?skipgram (context2vec substitutes in skipgram space plus skipgram context words)> <weight:{1}> <eval_data> <gpu> <w2salience> "  .format (sys.argv[0],WEIGHT_DICT.items())
#             sys.exit(1)
        
        model_param_file = args.model_param_file
        model_type=args.model_type
        
        if '-' in args.weight:
            weight,n_result=args.weight.split('-')
            weight=WEIGHT_DICT[int(weight)]
            n_result=int(n_result)
        else:
            weight=WEIGHT_DICT[int(args.weight)]
            n_result=20 #default is 20 top
            
        data =args.data
        gpu=args.gpu
        w2salience_f=args.w2salience_f
       
    
    #gpu setup 
   

    if gpu >= 0:
        cuda.check_cuda_available()
        cuda.get_device(gpu).use()    
    xp = cuda.cupy if gpu >= 0 else np
    
    # logging
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    
    #choose model type
    print ('read model....')
    if model_type=='context2vec':
        #read in model
        
        model_reader = ModelReader(model_param_file,gpu)
        w = xp.array(model_reader.w)
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        w_target=None
        word2index_target=None
        index2word_target=None
        
    elif model_type=='skipgram':
        model_w2v = gensim.models.Word2Vec.load(model_param_file)
        w=xp.array(deepcopy(model_w2v.wv.vectors))
        #vector normalize for target w embedding, consistent with context2vec w and convenient for cosine computation among substitutes
        s = xp.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model_w2v.wv.index2word
        word2index={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
        w_target=None
        word2index_target=None
        index2word_target=None
        
        print ('filter words for context....')

        model=rm_stopw_context(model_w2v)
        
    elif model_type=='context2vec-skipgram':
        model_param_context,model_param_w2v=model_param_file.split('?')
        model_reader = ModelReader(model_param_context,gpu)
        w = xp.array(model_reader.w)
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        
        model_w2v = gensim.models.Word2Vec.load(model_param_w2v)
        w_target=xp.array(model_w2v.wv.vectors)
        index2word_target=model_w2v.wv.index2word
        word2index_target={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
    
    elif model_type=='context2vec-skipgram?skipgram':
        model_param_context,model_param_w2v=model_param_file.split('?')
        #context2vec-skipgram
        model_reader = ModelReader(model_param_context,gpu)
        w = xp.array(model_reader.w)
        index2word = model_reader.index2word
        word2index =model_reader.word2index
        model = model_reader.model
        
        model_w2v = gensim.models.Word2Vec.load(model_param_w2v)
        w_target=xp.array(model_w2v.wv.vectors)
        index2word_target=model_w2v.wv.index2word
        word2index_target={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
    
        # skigpram
        model_skipgram = model_w2v
        w_skipgram=xp.array(deepcopy(model_skipgram.wv.vectors))
        #vector normalize for probe w embedding
        s = xp.sqrt((w_skipgram * w_skipgram).sum(1))
        s[s==0.] = 1.
        w_skipgram /= s.reshape((s.shape[0], 1))
        
        index2word_skipgram=model_skipgram.wv.index2word
        word2index_skipgram={key: model_skipgram.wv.vocab[key].index for key in model_skipgram.wv.vocab}
        w_target_skipgram=None
        word2index_target_skipgram=None
        index2word_target_skipgram=None
        
        print ('filter words for context....')

        model_skipgram=rm_stopw_context(model_skipgram)
        
                
        
    
    #remove stop words in target word space and asarray
    print ('filter words for target....')
    w,word2index,index2word=filter_w(w,word2index,index2word)
    if  index2word_target!=None:
        w_target,word2index_target,index2word_target=filter_w(w_target,word2index_target,index2word_target)
    if model_type=='context2vec-skipgram?skipgram':
        w_skipgram,word2index_skipgram,index2word_skipgram=filter_w(w_skipgram,word2index_skipgram,index2word_skipgram)
    
    #per word weight
    
    w2salience=None
    if weight==LDA:
        print ('load vectors and entropy')
        w2salience=pickle.load(open(w2salience_f))
    elif weight==INVERSE_W_FREQ:
        print ('load w2freq')
        w2salience=load_w2salience(w2salience_f,weight)
    elif weight==INVERSE_S_FREQ:
        print ('load w2freq')
        w2salience=load_w2salience(w2salience_f,weight)


    #combine parameters for skipgram?context2vec-skipgram
    if model_type=='context2vec-skipgram?skipgram':
        model=(model,model_skipgram)
        w=(w,w_skipgram)
        index2word=(index2word,index2word_skipgram)
        word2index=(word2index,word2index_skipgram)
        weight=(weight,WEIGHT_DICT[0])#assume that skipgram has no weight
        w2salience=(w2salience,w2salience)
        w_target=(w_target,w_target_skipgram)
        word2index_target=(word2index_target,word2index_target_skipgram)
        index2word_target=(index2word_target,index2word_target_skipgram)
    
    print (model_param_file,model_type,weight,data,w2salience_f)


read model....
Reading config file: ../models/context2vec/model_dir/MODEL-wiki.params.14
Config:  {'config_path': '../models/context2vec/model_dir/', 'model_file': 'MODEL-wiki.14', 'deep': 'yes', 'drop_ratio': '0.0', 'words_file': 'WORDS-wiki.targets.14', 'unit': '400'}
filter words for target....
('../models/context2vec/model_dir/MODEL-wiki.params.14?../models/wiki_all.model/wiki_all.sent.split.model', 'context2vec-skipgram', False, './eval_data/data-chimeras/dataset.l2.fixed.test.txt.punct', None)


In [136]:
    #read in data
    import time
    start_time = time.time()
    
    
    if data.split('/')[-2]== 'data-chimeras':

            eval_chimera(data,model,model_type,n_result,w,index2word,word2index,weight,w2salience,w_target,word2index_target,index2word_target)

    elif data.split('/')[-2]== 'data-nonces':
            ranks=eval_nonce(data,model,model_w2v,model_type,n_result,w,index2word,word2index,weight,w2salience,w_target,word2index_target,index2word_target)
    print("--- %s seconds ---" % (time.time() - start_time))

./eval_data/data-chimeras
2
canned sardines and  ___  between two slices of wholemeal bread and thinly spread flora original .
<module 'cupy' from '/usr/local/lib/python2.7/dist-packages/cupy/__init__.pyc'>
producing top 20 simwords
(<type 'cupy.core.core.ndarray'>, <type 'cupy.core.core.ndarray'>)
('context_embed original', array([-2.0292675 , -0.12115708, -0.5854739 , -0.2508919 , -1.0164323 ,
       -0.462306  , -0.4165849 ,  0.7679413 ,  0.2772599 ,  0.72865736],
      dtype=float32))
('context_embed_out', array([-0.1021727 , -0.04791794,  0.12457735, -0.15908252,  0.28662088,
       -0.04613013,  0.04267992, -0.04619194,  0.18829176,  0.36753569]))
('3', <type 'cupy.core.core.ndarray'>)
('weight mode', False)
('context_embedtype', <type 'cupy.core.core.ndarray'>)
erm ,  ___  , low fat dairy products , incidents of heart disease for those who have an olive oil rich diet .
<module 'cupy' from '/usr/local/lib/python2.7/dist-packages/cupy/__init__.pyc'>
producing top 20 simwords
(<typ

editorgiven1: 2.238339811034014
editorsurname4: 2.203155271444537
publicationdate: 2.1335763636014127
editorsurname3: 2.1155131776653233
editorsurname2: 2.105996202409689
editorsurname1: 2.096704571275271
laysummary: 2.0811817507098214
laydate: 2.0017722921305205
fireplace: 1.9740836982042487
cupboard: 1.9728704144423237
bathroom: 1.9618155681571665
ifeq: 1.9306907325212521
nightgown: 1.9279348146664572
cupboards: 1.916404930227175
mantelpiece: 1.8820554760943065
waistcoat: 1.8779508653809116
blouse: 1.876211071552982
napkin: 1.86794772300029
bookcase: 1.8630059083326977
rug: 1.8561534901122174
('context avg out', array([ 0.10269857,  0.10703545,  0.0314152 , -0.15105081,  0.15187912,
       -0.04278371, -0.0526107 , -0.03905508,  0.16288606,  0.05099644]))
('probes', ['stove', 'microwave', 'kettle', 'cage', 'wastebin', 'leopard'])
('gold', ['2.14', '1.57', '1.43', '1.71', '2', '2.14'])
('model_predict', [0.5238005371844539, 0.28699749426396876, 0.3068943115544493, 0.32809485701461355,

oboes: 2.750931188208136
bassoons: 2.654600312240229
clarinets: 2.537596698840725
trombones: 2.481426538541236
timpani: 2.4532863930364197
glockenspiel: 2.4344695196224198
synthesizers: 2.4135247025476394
cymbals: 2.3977570220332516
contrabassoon: 2.3459155955309576
saxophones: 2.3402201512810334
continuo: 2.3231693449584014
synthesisers: 2.3201009887090573
synthesizer: 2.3139067823892043
cellos: 2.306330359472588
synths: 2.3056980668325036
12-string: 2.3041622177243344
arpeggios: 2.295027385328536
flutes: 2.294414134594403
violas: 2.290939302484384
synthesiser: 2.277272922211865
('context avg out', array([-0.0772147 ,  0.02780987,  0.01864267,  0.03637049, -0.03602265,
       -0.10516457, -0.11318388,  0.06652762,  0.18946293,  0.1467604 ]))
('probes', ['bagpipe', 'harmonica', 'whistle', 'shotgun', 'bear', 'bouquet'])
('gold', ['1.57', '1.29', '2.43', '1.71', '1.29', '1.29'])
('model_predict', [0.4968860323258222, 0.6085543948473886, 0.4302822243838753, 0.28629131783096445, 0.11940580

opossums: 2.493683613087039
porcupines: 2.4477639142238456
egrets: 2.4261764192795745
ibises: 2.423867476308466
muntjac: 2.3980984303660806
partridges: 2.397943602063762
ungulates: 2.392352552137471
quails: 2.3914227589490586
avocets: 2.389604917045979
tapirs: 2.367809572012667
passeriformesfamily: 2.3640126970105055
possums: 2.3551988956252954
mongooses: 2.3533523988081027
woodpeckers: 2.3453523399645264
cormorants: 2.343172769848377
raccoons: 2.3297902917275537
peafowl: 2.3235010315538327
spoonbills: 2.3228077980804533
herons: 2.3173429032442305
langur: 2.296816664212633
('context avg out', array([ 0.00618237,  0.3478065 , -0.03353576, -0.23868313,  0.19271689,
       -0.03105689, -0.12059154, -0.11871297,  0.11046025,  0.14215927]))
('probes', ['caribou', 'groundhog', 'hare', 'spider', 'catapult', 'bolt'])
('gold', ['2.86', '2.29', '2.57', '2.71', '1.57', '1.71'])
('model_predict', [0.46320878170127844, 0.4293346161180177, 0.43816230442897985, 0.45340429224277534, 0.252110008141565,

nudibranch: 2.299538267203866
mollusk: 2.2781701576517617
theropod: 2.2451127640682307
gastropod: 2.240772392238052
quadrupedal: 2.2143889982687455
tapir: 2.209627108452894
sauropod: 2.1965800119094863
beaked: 2.1764140506550214
plesiosaur: 2.1756757325129152
dorid: 2.1737489299997903
avocets: 2.1669322921525005
theropods: 2.1634779469456573
forelimbs: 2.1623254014114703
herbivorous: 2.1540252976831034
pterosaur: 2.149037655448951
prehensile: 2.1480415820454244
flightless: 2.146775981961759
mollusks: 2.142131371370871
snouted: 2.138717930003229
blenny: 2.13532058908887
('context avg out', array([-0.06970209,  0.19636719, -0.05136029, -0.16331853,  0.21055678,
       -0.11647631, -0.10055953, -0.08797573,  0.09059065,  0.06088054]))
('probes', ['caribou', 'groundhog', 'hare', 'spider', 'catapult', 'bolt'])
('gold', ['2.71', '2.14', '2.71', '3', '1.71', '1.43'])
('model_predict', [0.41864574650474756, 0.3763058625510568, 0.3587458169886524, 0.500040567890686, 0.31601645000906575, 0.31133

...

tailskid: 2.167957688528462
axles: 2.1665352726703144
gearbox: 2.122147822048838
5-speed: 2.1160284876121764
tailwheel: 2.1132810267882443
bogies: 2.10696330286651
4-speed: 2.095463875380573
axle: 2.0866111994913856
undercarriage: 2.067289745101032
seats-in-side-by-side: 2.0643631914440883
camshaft: 2.0551331653463647
driveshaft: 2.0538173103142574
mainwheels: 2.0524090903755643
6-speed: 2.0508684349893063
brakes: 2.0472042037568174
smokebox: 2.0458578910804057
nosewheel: 2.0438641696996784
seats-in-tandem: 2.03633687272715
nacelles: 2.0266646972269107
wheel-drive: 2.025386871092727
('context avg out', array([ 0.0417168 ,  0.05039442, -0.0399619 , -0.05470607, -0.12775457,
        0.0446965 , -0.04309306, -0.00458444,  0.0865627 ,  0.03685168]))
('probes', ['skateboard', 'jeep', 'train', 'fridge', 'shed', 'parakeet'])
('gold', ['1.71', '2.29', '2', '1.57', '2.43', '1.86'])
('model_predict', [0.3505168781381252, 0.524767378932637, 0.4875376869344964, 0.3770466265489803, 0.39933374960727

larvae: 2.6370462979265503
pupate: 2.5337112000235154
axils: 2.4804413976286646
pupae: 2.4504131133659364
conidia: 2.4007923452655398
catkins: 2.3839668959787907
instars: 2.3822291443838632
florets: 2.370040602524962
petioles: 2.3395369169159617
hyphae: 2.3390690490791126
inflorescences: 2.3332587232868125
spores: 2.3172038419801417
sepals: 2.3107337339140654
pupation: 2.3057697344038424
bracts: 2.2904201262297237
frass: 2.2732008307806835
sporangia: 2.2694107399492944
pelecaniformesfamily: 2.2654343462768787
pupal: 2.2639921786375856
forewings: 2.2541916953035006
('context avg out', array([ 0.05788613,  0.16899088, -0.13546716, -0.14618289,  0.04904069,
       -0.03904393, -0.13148694,  0.02159973, -0.06207357,  0.15858028]))
('probes', ['beetle', 'grasshopper', 'spider', 'shack', 'porcupine', 'crane'])
('gold', ['2.71', '2.86', '2.43', '2.43', '1.86', '2.14'])
('model_predict', [0.42447422386335, 0.32389178973563876, 0.34682659837753727, 0.2995786366257789, 0.291033397824613, 0.26335

oboes: 2.856201351748124
bassoons: 2.7326891906702273
trombones: 2.624497520820473
clarinet: 2.5857565128484383
timpani: 2.5505353613633743
clarinets: 2.5438160792737836
continuo: 2.5383562837373788
trombone: 2.4822388039953616
bassoon: 2.4655671823092815
vibraphone: 2.462944029182653
woodwinds: 2.443590599548553
flugelhorn: 2.4435216040789944
saxophone: 2.428463645237871
contrabassoon: 2.4171905167834735
woodwind: 2.4132323537122726
glockenspiel: 2.386809060888064
saxophones: 2.3771108442246884
oboe: 2.3647992693245996
contrabass: 2.3625429945730163
violas: 2.3556693492166616
('context avg out', array([-0.13254281, -0.0998426 ,  0.13756721, -0.0245419 ,  0.03336055,
       -0.23260357, -0.06274331,  0.06310956,  0.14591085,  0.10466659]))
('probes', ['harmonica', 'drum', 'racquet', 'cabin', 'house', 'bolt'])
('gold', ['2.57', '2.43', '1.29', '1.43', '1.86', '2.29'])
('model_predict', [0.7384865110562483, 0.588399222969437, 0.3070530541516182, 0.19500877118457827, 0.17528049162068676, 

oboes: 2.612755648459187
continuo: 2.538881603142491
bassoons: 2.4916668387228023
sonatas: 2.4051626106105495
concertos: 2.404122259977587
concertante: 2.317909975988541
motets: 2.30887667893258
trombones: 2.301670501674011
timpani: 2.292003990427395
arpeggios: 2.280844467431914
concerti: 2.2796676794140884
fugues: 2.2647338998645243
cantatas: 2.263003225443118
clarinets: 2.2612835176974135
octaves: 2.259430693931264
ostinato: 2.258810196510061
soloists: 2.2534585209053204
editorgiven1: 2.2495201862156358
concerto: 2.2465762468943318
cellos: 2.239611762202014
('context avg out', array([-0.06877351, -0.03303879, -0.01085216, -0.01295457,  0.05125404,
       -0.17068099,  0.0835807 ,  0.13882613,  0.00982652,  0.00227037]))
('probes', ['banjo', 'gorilla', 'whistle', 'worm', 'dress', 'pine'])
('gold', ['2.43', '2.43', '1.71', '1.71', '1.57', '1.14'])
('model_predict', [0.5191704899686589, 0.23440859960179666, 0.3580785303103172, 0.22287848416371686, 0.3025958880416212, 0.170946072083268])

KeyboardInterrupt: 